In [27]:
"""Implements classes and methods for automated scalping.

Description
----------
Implements classes to represent market structure, generate trade
signals and execute them on an exchange by communicating via API.

Classes
----------
    BinanceClient:
        Provides an interface for the trading bot to communicate
        with the Binance exchange via its' API.

    MarketStructure:
        Represents the market structure of a crypto coin
        at one point in time.
    
    Trade:
        Implements generic properties of trades that are
        common amongst all particular trading strategies
        such as buying and selling.
    
    ContinuationTrade: Inherits from Trade
        Implements the logical rules for a trend continuation trade
        triggered by an initial confirmation of trend continuation.

Functions
----------
    backtest:
        Goes through a give set of historical data and applies the
        trading strategy to this data, tracking results.

Exceptions
----------
    Exports no exceptions.
"""

%matplotlib inline 

# data analysis
import numpy as np
import pandas as pd
import pandas_ta
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn import linear_model 

# data communication
import configparser
from typing import Optional, Dict, Any
from requests import Session, Request, Response

# utilities
import math
import logging
import apscheduler
from apscheduler.schedulers.blocking import BlockingScheduler
import unittest
import calendar
from datetime import datetime, timedelta

In [28]:
class BinanceClient:
    """Provides an interface for the trading bot to communicate with the Binance exchange via its' API."""
    
    def __init__(self, endpoint):
        self.maker_fees_USD_futures = 0.0002
        self.taker_fees_USD_futures = 0.0004
        self.session = None
        self.config = None
        self.api_key = None
        self.api_secret = None
        self.endpoint = endpoint
        
        
    def open_session(self):
        """Opens a session with the Binance server."""
        
        self.session = Session()
        
        
    def read_config(self):
        """Initializes the Binance client."""
        
        self.config = configparser.ConfigParser()
        self.config.read('config.ini')
        
        self.api_key = self.config['BINANCE API']['api_key']
        self.api_secret = self.config['BINANCE API']['api_secret']
        
        
    def process_response(self, response: Response) -> Any:
        """Processes the response the server sends to the clients request."""
        
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            return data
    
    
    def sign_request(self, request: Request) -> None:
        """Signs confidential requests that need the users API key and API secret."""
        
        ts = int(time.time() * 1000)
        request.params['timestamp'] = str(ts)
        signature_payload = urlencode(request.params).encode("utf-8")
        signature = hmac.new(BINANCE_API_SECRET.encode("utf-8"), signature_payload, hashlib.sha256).hexdigest()
        request.headers['X-MBX-APIKEY'] = self.api_key
        request.params['signature'] = signature
        
        
    def request(self, method: str, path: str,**kwargs,) -> Any:
        """Executes a non-confidential request to the Binance server."""
        
        request = Request(method, self.endpoint + path, **kwargs)
        response = self.session.send(request.prepare())
        return self.process_response(response)
    
    
    def signed_request(self, method: str, path: str, params, **kwargs) -> Any:
        """Executes a confidential request to the Binance server"""
        
        request = Request(method, self.endpoint + path, params=params, **kwargs)
        self.sign_request(request)
        response = session.send(request.prepare())
        return self.process_response(response)
        
        
    def get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        """Executes an HTTPS GET request."""
        
        return self.request('GET', path, params=params)
    
    
    def post(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        """Executes an HTTPS POST request."""
        
        return self.signed_request('POST', path, params=params)
    
    
    def delete(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        """Executes an HTTPS DELETE request."""
        
        return self.signed_request('DELETE', path, json=params)
    
    
    def place_order(self, market: str, side: str, size: float, reduce_only: bool = False,
                    order_type: str = 'MARKET') -> dict:
        """Places a buy or sell order with the exchange."""
        return post(self, 'fapi/v1/order', {
            'symbol': market,
            'side': side,
            'type': order_type,
            'quantity': size,
            "reduceOnly": reduce_only
        })

In [29]:
class MarketStructure:
    """Represents the market structure of a crypto coin at one point in time."""
    
    def __init__(self, prev_high: tuple, prev_low: tuple, 
                 provisional_high: tuple, provisional_low: tuple,
                 recent_break: bool = False):
        self.msb = False
        self.stay_in_range = False
        self.continuation = False
        self.trend = False
        self.prev_high = prev_high
        self.prev_low = prev_low
        self.provisional_high = provisional_high
        self.provisional_low = provisional_low
        self.recent_break = recent_break
        
        
    def _break_up_trend(self, row):
        """Sets the new market structure after break of an up trend."""
        
        self.trend = False
        self.prev_high = self.provisional_high
        self.prev_low = tuple(row[['low', 'open time']])
        self.provisional_low = tuple(row[['low', 'open time']])
        logging.info('Up Trend Broken')
        
        
    def _break_down_trend(self, row):
        """Sets the new market structure after break of a down trend."""
        
        self.trend = True
        self.prev_high = tuple(row[['high', 'open time']])
        self.prev_low = self.provisional_low
        self.provisional_high = tuple(row[['high', 'open time']])
        logging.info('Down Trend Broken')
        
        
    def _continue_up_trend(self, row):
        """Sets the new market structure after an up trend continues."""
        
        self.prev_low = self.provisional_low
        self.prev_high = tuple(row[['high', 'open time']])
        self.provisional_high = tuple(row[['high', 'open time']])
        logging.info('Continuing Up Trend')
        
        
    def _continue_down_trend(self, row):
        """Sets the new market structure after a down trend continues."""
        
        self.prev_high = self.provisional_high
        self.prev_low = tuple(row[['low', 'open time']])
        self.provisional_low = tuple(row[['low', 'open time']])
        logging.info('Continuing Down Trend')
        
    def _stay_in_range(self, row):
        """Keeps track of provisional information while range bound."""
        
        if row['close'] - row['open'] >= 0:
            self.provisional_high = tuple(row[['high', 'open time']])
        if row['close'] - row['open'] < 0:
            self.provisional_low = tuple(row[['low', 'open time']])
        logging.info('Staying in Range')
        
        
    def next_candle(self, row):
        """Investigates how the market structure changes with the next incoming price data candle."""
        
        close = row['close']
        self.msb = False
        self.continuation = False
        self.stay_in_range = False
        trend = self.trend
        
        if close > self.prev_high[0]:
            if self.trend == False:
                self._break_down_trend(row)
                self.msb = True
            else:
                self._continue_up_trend(row)
                self.continuation = True
        elif close < self.prev_low[0]:
            if self.trend == False:
                self._continue_down_trend(row)
                self.continuation = True
            else:
                self._break_up_trend(row)
                self.msb = True
        else:
            self._stay_in_range(row)
            self.stay_in_range = True
            
        return trend, self.msb, self.continuation, self.stay_in_range

In [30]:
class MachineLearning():
    
    def __init__(self):
        pass

In [31]:
class LogisticRegression(MachineLearning):
    
    def __init__(self):
        super(MachineLearning, self).__init__()
    
    def regress(self):
        
        #X represents the size of a tumor in centimeters.
        X = np.array([3.78, 2.44, 2.09, 0.14, 1.72, 1.65, 4.92, 4.37, 4.96, 4.52, 3.69, 5.88]).reshape(-1,1)

        #Note: X has to be reshaped into a column from a row for the LogisticRegression() function to work.
        #y represents whether or not the tumor is cancerous (0 for "No", 1 for "Yes").
        y = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]) 
        logr = linear_model.LogisticRegression()
        logr.fit(X,y)
        #predict if tumor is cancerous where the size is 3.46mm:
        predicted = logr.predict(np.array([3.46]).reshape(-1,1))
        print(predicted)

In [40]:
class Trade:
    """Implements generic properties of trades that are common amongst all particular trading strategies
       such as buying and selling."""
    
    def __init__(self, bc: BinanceClient, ms: MarketStructure, ml: MachineLearning,
                 equity: float, max_risk: float, max_leverage: float = 1.0):
        self.ml = ml
        self.ms = ms
        self.bc = bc
        self._equity = equity
        self._max_risk = max_risk
        self._max_leverage = max_leverage
        self.equity_curve = [equity]
        self.position_size = [0]
        self.position = 0
        self.num_trades = 0
        self.wins = 0
        self.win_rate = 0
        self.kelly = 0.02
        self.open = 0
        self.close = 0
        self.long_trigger = False
        self.long_position = False
        self.short_trigger = False
        self.short_position = False
        self.entry = None
        self.stop_loss = None
        self.target = None

        
    @property
    def equity(self):
        """Returns the current equity value of the strategy."""
        
        if self.long_position or self.short_position:
            return (self.position*self.close) + self._equity
        else:
            return self._equity
    
    
    def plot_equity_curve(self):
        """Plots the equity curve of the strategy."""
        
        plt.plot(self.equity_curve)
        plt.show()
        
        
    def plot_position_size(self):
        """Plots the position size of the strategy over time."""
        
        plt.plot(self.position_size)
        plt.show()
        
    
    def predict(self):
        """Uses Machine Learning to predict if a trade signal will be profitable."""
        
        self.ml.regress()
        
        
    def long(self, price, risk):
        """Enters a long trade."""
        
        #self.kelly = max(0.001, 1.5*self.win_rate - 0.5)
        #print('win rate: ', self.win_rate)
        #print('kelly: ', self.kelly)
        #max_risk = min(self._max_risk, self.kelly)
        trade_size = min(self._max_leverage*self._equity, 
                         (self._max_risk/risk) * self._equity)
        coins = ((1.-self.bc.taker_fees_USD_futures)*trade_size)/price
        self.position += coins
        self._equity -= trade_size
        logging.info('Entered New Long Position')
        
        
    def short(self, price, risk):
        """Enters a short trade."""
        
        #self.kelly = max(0.001, 1.5*self.win_rate - 0.5)
        #print('win rate: ', self.win_rate)
        #print('kelly: ', self.kelly)
        #max_risk = min(self._max_risk, self.kelly)
        trade_size = min(self._max_leverage*self._equity, 
                         (self._max_risk/risk) * self._equity)
        coins = ((1-self.bc.taker_fees_USD_futures)*trade_size)/price
        self.position -= coins
        self._equity += (1.-self.bc.taker_fees_USD_futures) * trade_size
        logging.info('Entered New Short Position')
        
        
    def close_trade(self, price):
        """Closes an open long or short positoin."""
        
        cash = self.position*price
        self._equity += (1.-self.bc.taker_fees_USD_futures) * cash
        self.position = 0

In [41]:
bc = BinanceClient(endpoint = 'https://fapi.binance.com/')
bc.open_session()
bc.read_config()

In [42]:
ml = LogisticRegression()

In [43]:
btc = {'market_name': 'BTCBUSD', 'ath': (58434.0, '2021-02-21 19:00:00+00:00'), 
      'prev_low': (57465.0, '2021-02-21 18:00:00+00:00'),
      'start': datetime(2021, 2, 21, 20, 0, 0, 0), 'initial_equity': 100, 'timeframe': '1h',
      'max risk': 0.1, 'leverage': 1.0, 'r/r': 2.0}

In [44]:
market = btc

market_name = market['market_name']
ath = market['ath']
prev_low = market['prev_low']
start = market['start']
initial_equity = market['initial_equity']
timeframe = market['timeframe']
    
rr = market['r/r']
leverage = market['leverage']
risk = market['max risk']
                
ms = MarketStructure(ath, prev_low, ath, prev_low)

tr = Trade(bc, ms, ml, initial_equity, risk, leverage)

In [45]:
tr.predict()

[0]
